### Section 2.1

To someone who does not know (or needs a quick recap) section 2.1 gives a summary of what word embeddings are.

### Section 2.2 (Tokenizing text)


We start coding from section 2.2 that explains how we tokenize text.
We download all the text from 'The verdict', available on WikiSource.


In [1]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)




This document is a concise text file containing just over 20,000 characters, which should be sufficient for building a basic model. The primary goal is to tokenize all the words in this document as a foundational step in the model-building process.

In [2]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))

Total number of character: 20479


We remove white spaces and generate 'tokens' (containing words and special characters).

In [3]:
import re

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

### Section 2.3 (Converting tokens into token IDs)

Create 'vocabulary' from 'tokens'

In [4]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print("Total number of unique tokens:", vocab_size)

Total number of unique tokens: 1130


Assign a unique integer for every token

In [5]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- We create a `Tokenizer` class with the following two primary methods:
  - **`encode`**: Converts a token into its corresponding token ID.
  - **`decode`**: Converts a token ID back into its original token.
- This class serves as a foundational utility for tokenizing text and reconstructing it from token IDs.


In [6]:
class Tokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [7]:
tokenizer = Tokenizer(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""

In [8]:
ids = tokenizer.encode(text)
print(ids)
tokenizer.decode(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

### Section 2.4 (Adding special context tokens)

We add 2 additional token: <|unk|> (representing words not part of the vocab) and <|endoftext|> (Seperates two unrelated text source)

In [9]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

Updated tokenizer

In [10]:
class Tokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

### Section 2.5 (BytePair encoding)

Byte Pair Encoding (BPE) is a subword tokenization algorithm commonly used to split text into smaller units (subwords) that strike a balance between words and individual characters. This approach is especially effective for handling rare or unseen words, making it widely used in modern language models like GPT and BERT.

We use `tiktoken`, OpenAI's open-source library. (It is 5 times faster! 😮)

In [11]:
!python --version

Python 2.7.16


In [12]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.8.0


Below lines is equivalent of encode method of Tokenizer class implemented above.
someunknownPlace is not a word in the vocabulary, but it is assigned a token. There is a token for every unknown word.

In [13]:
tokenizer = tiktoken.get_encoding("gpt2")

text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [14]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


### Section 2.6 (Data sampling with a sliding window)

We now want to create input->target data that will be used in training.
BPE returns 5145 tokens.

In [15]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


Below code is for demonstration of source and terget for a small section of the text (1st 50 tokens).

In [16]:
enc_sample = enc_text[50:]

Below x represents input tokens and y represents targets, which are inputs shifted by 1.

In [17]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


Left of arrow (---->): Input to the LLM  
Right of arrow : Target token ID.

In [18]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)
          
print("============================")
    
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257
 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


We now implement a Dataloader using pytorch to prepare for model training.
Inputs and targets are used as tensors

In [19]:
#Implementation of the dataloader

import torch
from torch.utils.data import Dataset, DataLoader


class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the text into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


/Users/bhavtosh/env1/lib/python3.9/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


create_dataloader_v1() below loads the input using data loader

In [20]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    """
    Args:
        txt (str): The input text to be tokenized and processed into a dataset.
        batch_size (int, optional): The number of samples per batch. Defaults to 4.
        max_length (int, optional): The maximum token length for each sequence. Defaults to 256.
        stride (int, optional): The stride for generating overlapping sequences. Defaults to 128.
        shuffle (bool, optional): Whether to shuffle the dataset before each epoch. Defaults to True.
        drop_last (bool, optional): Whether to drop the last batch if it's smaller than the batch size. Defaults to True.
        num_workers (int, optional): Number of worker threads for data loading. Defaults to 0.

    Returns:
        DataLoader: A PyTorch DataLoader for iterating over the tokenized dataset.

    """
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDataset(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader


Below we test the dataloader with a batch size of 1 for an LLM with a context size (max_length) of 4.  
'stride': no. of positions the input shifts across batches.


In [21]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


Example with stride=2

In [22]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=2, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[2885, 1464, 1807, 3619]]), tensor([[1464, 1807, 3619,  402]])]


In future, we can increase the stride and the batch simultaneously to prevent overfitting.  
If batch size is too small there is noisy model updates during training.  
Below is an example of batch_size=4 and stride=4

In [23]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(
    raw_text, batch_size=4, max_length=4, stride=4, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)
second_batch = next(data_iter)
print(second_batch)

[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922]])]
[tensor([[ 922, 5891, 1576,  438],
        [ 568,  340,  373,  645],
        [1049, 5975,  284,  502],
        [ 284, 3285,  326,   11]]), tensor([[5891, 1576,  438,  568],
        [ 340,  373,  645, 1049],
        [5975,  284,  502,  284],
        [3285,  326,   11,  287]])]


### Section 2.7 (Creating token embeddings)

Last stage is to create embeddings for the tokens.
Below is an example where we create embeddings for a vocalbulary of size 6 and emb dimensions is 3.

In [26]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


embedding_layer is a lookkup operation

In [28]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


### Section 2.8 (Encoding word positions)

## Positional Information in Tokens

To encode positional information within the text, we use the following formula:

**Input Embedding = Token Embedding + Positional Embedding**

---

### Shortcoming of Token Embeddings
The current implementation of token embeddings has a limitation:  
**It lacks positional information.**

---

### Types of Positional Information
1. **Absolute Positioning**:  
   Tokens are embedded with information about their exact position in the sequence.

2. **Relative Positioning**:  
   The model learns embeddings based on how far apart tokens are from each other, rather than their exact positions.

---

### Focus on Relative Positioning
Relative positioning focuses on the **distance between tokens** rather than their exact positions. This approach is particularly useful in capturing contextual relationships between tokens.

---

### GPT Models and Positional Embedding
GPT models use **absolute positional encoding** (PE).


We use emb dimension = 256

In [29]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [31]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

Batch contains 8 examples with 4 tokens each

In [32]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [33]:
print("Target IDs:\n", targets)
print("\Targets shape:\n", targets.shape)

Target IDs:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
\Targets shape:
 torch.Size([8, 4])


Tensor representation for the batch

In [34]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


**Creating Positional Embedding**   
It is just another embedding layer.


In [35]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [36]:
pos_embeddings

tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<EmbeddingBackward0>)

In [39]:
token_embeddings.shape


torch.Size([8, 4, 256])

Add pos_embeddings to every batch token embeddings  

In [41]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


In [ ]:
A major takeaway is that as part of the data preparation, we need to add postional encoding with the token embedding